# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

In [5]:
n_features

296

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1051 entries, 0 to 1050
Columns: 296 entries, MSSubClass to SaleCondition_Partial
dtypes: float64(296)
memory usage: 2.4 MB


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [11]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 1s 8ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [13]:
# Numeric column names
numeric_columns = X_train_numeric.columns

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [14]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [15]:
# Train the model 
normalized_input_model.fit(X_train, 
                           y_train, 
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 1s 7ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [16]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [18]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 7ms/step - loss: 0.3491 - mse: 0.3491 - val_loss: 0.1773 - val_mse: 0.1773
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.2412 - mse: 0.2412 - val_loss: 0.3460 - val_mse: 0.3460
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: 0.2098 - mse: 0.2098 - val_loss: 0.1415 - val_mse: 0.1415
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1751 - mse: 0.1751 - val_loss: 0.1424 - val_mse: 0.1424
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1553 - mse: 0.1553 - val_loss: 0.1304 - val_mse: 0.1304
Epoch 6/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1345 - mse: 0.1345 - val_loss: 0.1249 - val_mse: 0.1249
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1346 - mse: 0.1346 - val_loss: 0.1215 - val_mse: 0.1215
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1258 - m

33/33 [==============================] - 0s 4ms/step - loss: 0.0250 - mse: 0.0250 - val_loss: 0.1125 - val_mse: 0.1125
Epoch 64/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0246 - mse: 0.0246 - val_loss: 0.1135 - val_mse: 0.1135
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0239 - mse: 0.0239 - val_loss: 0.1129 - val_mse: 0.1129
Epoch 66/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0238 - mse: 0.0238 - val_loss: 0.1121 - val_mse: 0.1121
Epoch 67/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0232 - mse: 0.0232 - val_loss: 0.1124 - val_mse: 0.1124
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0227 - mse: 0.0227 - val_loss: 0.1130 - val_mse: 0.1130
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0222 - mse: 0.0222 - val_loss: 0.1118 - val_mse: 0.1118
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0224 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1186 - val_mse: 0.1186
Epoch 126/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1173 - val_mse: 0.1173
Epoch 127/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1177 - val_mse: 0.1177
Epoch 128/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0107 - mse: 0.0107 - val_loss: 0.1181 - val_mse: 0.1181
Epoch 129/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1182 - val_mse: 0.1182
Epoch 130/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1185 - val_mse: 0.1185
Epoch 131/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1173 - val_mse: 0.1173
Epoch 132/150
33/33 [==============================] - 0s 3ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0079 - mse: 0.0079


[0.007933397777378559, 0.007933397777378559]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [23]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.1180 - mse: 0.1180


[0.11802662909030914, 0.11802662909030914]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [24]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

26997.372971956283

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [25]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,), kernel_initializer='he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 7ms/step - loss: 0.4396 - mse: 0.4396 - val_loss: 0.1886 - val_mse: 0.1886
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.2456 - mse: 0.2456 - val_loss: 0.1636 - val_mse: 0.1636
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: 0.2063 - mse: 0.2063 - val_loss: 0.1370 - val_mse: 0.1370
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1893 - mse: 0.1893 - val_loss: 0.1293 - val_mse: 0.1293
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1702 - mse: 0.1702 - val_loss: 0.1805 - val_mse: 0.1805
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1532 - mse: 0.1532 - val_loss: 0.1225 - val_mse: 0.1225
Epoch 7/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1393 - mse: 0.1393 - val_loss: 0.1187 - val_mse: 0.1187
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1312 - m

33/33 [==============================] - 0s 4ms/step - loss: 0.0267 - mse: 0.0267 - val_loss: 0.1259 - val_mse: 0.1259
Epoch 64/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0258 - mse: 0.0258 - val_loss: 0.1254 - val_mse: 0.1254
Epoch 65/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0258 - mse: 0.0258 - val_loss: 0.1342 - val_mse: 0.1342
Epoch 66/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0257 - mse: 0.0257 - val_loss: 0.1259 - val_mse: 0.1259
Epoch 67/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0247 - mse: 0.0247 - val_loss: 0.1273 - val_mse: 0.1273
Epoch 68/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0245 - mse: 0.0245 - val_loss: 0.1279 - val_mse: 0.1279
Epoch 69/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0241 - mse: 0.0241 - val_loss: 0.1287 - val_mse: 0.1287
Epoch 70/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0238 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1286 - val_mse: 0.1286
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0114 - mse: 0.0114 - val_loss: 0.1288 - val_mse: 0.1288
Epoch 127/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0114 - mse: 0.0114 - val_loss: 0.1282 - val_mse: 0.1282
Epoch 128/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1290 - val_mse: 0.1290
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0111 - mse: 0.0111 - val_loss: 0.1281 - val_mse: 0.1281
Epoch 130/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1303 - val_mse: 0.1303
Epoch 131/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1288 - val_mse: 0.1288
Epoch 132/150
33/33 [==============================] - 0s 3ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [26]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0089 - mse: 0.0089


[0.008928673341870308, 0.008928673341870308]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [27]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.1278 - mse: 0.1278


[0.12779462337493896, 0.12779462337493896]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [28]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,), kernel_initializer='lecun_normal'))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 7ms/step - loss: 0.4320 - mse: 0.4320 - val_loss: 0.1685 - val_mse: 0.1685
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.2292 - mse: 0.2292 - val_loss: 0.1435 - val_mse: 0.1435
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1893 - mse: 0.1893 - val_loss: 0.1299 - val_mse: 0.1299
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1589 - mse: 0.1589 - val_loss: 0.1291 - val_mse: 0.1291
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1447 - mse: 0.1447 - val_loss: 0.1157 - val_mse: 0.1157
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1332 - mse: 0.1332 - val_loss: 0.1170 - val_mse: 0.1170
Epoch 7/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1165 - mse: 0.1165 - val_loss: 0.1227 - val_mse: 0.1227
Epoch 8/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1123 - m

33/33 [==============================] - 0s 4ms/step - loss: 0.0258 - mse: 0.0258 - val_loss: 0.1206 - val_mse: 0.1206
Epoch 64/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0244 - mse: 0.0244 - val_loss: 0.1183 - val_mse: 0.1183
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0236 - mse: 0.0236 - val_loss: 0.1268 - val_mse: 0.1268
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0237 - mse: 0.0237 - val_loss: 0.1175 - val_mse: 0.1175
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0228 - mse: 0.0228 - val_loss: 0.1170 - val_mse: 0.1170
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0227 - mse: 0.0227 - val_loss: 0.1186 - val_mse: 0.1186
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0227 - mse: 0.0227 - val_loss: 0.1172 - val_mse: 0.1172
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0230 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0115 - mse: 0.0115 - val_loss: 0.1217 - val_mse: 0.1217
Epoch 126/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1229 - val_mse: 0.1229
Epoch 127/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1225 - val_mse: 0.1225
Epoch 128/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1220 - val_mse: 0.1220
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.1234 - val_mse: 0.1234
Epoch 130/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1221 - val_mse: 0.1221
Epoch 131/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.1213 - val_mse: 0.1213
Epoch 132/150
33/33 [==============================] - 0s 4ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [29]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0087 - mse: 0.0087


[0.008663848973810673, 0.008663848973810673]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [30]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.1226 - mse: 0.1226


[0.12257157266139984, 0.12257157266139984]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [31]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', 
                      loss='mse', 
                      metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 7ms/step - loss: 0.3810 - mse: 0.3810 - val_loss: 0.1297 - val_mse: 0.1297
Epoch 2/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1683 - mse: 0.1683 - val_loss: 0.1142 - val_mse: 0.1142
Epoch 3/150
33/33 [==============================] - 0s 7ms/step - loss: 0.1414 - mse: 0.1414 - val_loss: 0.1071 - val_mse: 0.1071
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1141 - mse: 0.1141 - val_loss: 0.0907 - val_mse: 0.0907
Epoch 5/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0914 - mse: 0.0914 - val_loss: 0.0991 - val_mse: 0.0991
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0832 - mse: 0.0832 - val_loss: 0.1086 - val_mse: 0.1086
Epoch 7/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0723 - mse: 0.0723 - val_loss: 0.1043 - val_mse: 0.1043
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0571 - m

33/33 [==============================] - 0s 4ms/step - loss: 0.0085 - mse: 0.0085 - val_loss: 0.1065 - val_mse: 0.1065
Epoch 64/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0090 - mse: 0.0090 - val_loss: 0.1032 - val_mse: 0.1032
Epoch 65/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0074 - mse: 0.0074 - val_loss: 0.1086 - val_mse: 0.1086
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0085 - mse: 0.0085 - val_loss: 0.1182 - val_mse: 0.1182
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0063 - mse: 0.0063 - val_loss: 0.0995 - val_mse: 0.0995
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0081 - mse: 0.0081 - val_loss: 0.1078 - val_mse: 0.1078
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0068 - mse: 0.0068 - val_loss: 0.1187 - val_mse: 0.1187
Epoch 70/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0075 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0046 - mse: 0.0046 - val_loss: 0.0897 - val_mse: 0.0897
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0047 - mse: 0.0047 - val_loss: 0.0972 - val_mse: 0.0972
Epoch 127/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0045 - mse: 0.0045 - val_loss: 0.0915 - val_mse: 0.0915
Epoch 128/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0054 - mse: 0.0054 - val_loss: 0.0999 - val_mse: 0.0999
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0045 - mse: 0.0045 - val_loss: 0.0921 - val_mse: 0.0921
Epoch 130/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0045 - mse: 0.0045 - val_loss: 0.1113 - val_mse: 0.1113
Epoch 131/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0042 - mse: 0.0042 - val_loss: 0.0993 - val_mse: 0.0993
Epoch 132/150
33/33 [==============================] - 0s 3ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [32]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0026 - mse: 0.0026


[0.002582389861345291, 0.002582389861345291]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [33]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.1004 - mse: 0.1004


[0.10035508871078491, 0.10035508871078491]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [34]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam', 
                      loss='mse', 
                      metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 7ms/step - loss: 0.3696 - mse: 0.3696 - val_loss: 0.1315 - val_mse: 0.1315
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1830 - mse: 0.1830 - val_loss: 0.1044 - val_mse: 0.1044
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1167 - mse: 0.1167 - val_loss: 0.0945 - val_mse: 0.0945
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1018 - mse: 0.1018 - val_loss: 0.1015 - val_mse: 0.1015
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0777 - mse: 0.0777 - val_loss: 0.0878 - val_mse: 0.0878
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0562 - mse: 0.0562 - val_loss: 0.0934 - val_mse: 0.0934
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0414 - mse: 0.0414 - val_loss: 0.0885 - val_mse: 0.0885
Epoch 8/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0327 - m

Epoch 63/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0059 - mse: 0.0059 - val_loss: 0.0927 - val_mse: 0.0927
Epoch 64/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0052 - mse: 0.0052 - val_loss: 0.1043 - val_mse: 0.1043
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0040 - mse: 0.0040 - val_loss: 0.0959 - val_mse: 0.0959
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0039 - mse: 0.0039 - val_loss: 0.1042 - val_mse: 0.1042
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0032 - mse: 0.0032 - val_loss: 0.0942 - val_mse: 0.0942
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0022 - mse: 0.0022 - val_loss: 0.1011 - val_mse: 0.1011
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0023 - mse: 0.0023 - val_loss: 0.0979 - val_mse: 0.0979
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: 0.

33/33 [==============================] - 0s 5ms/step - loss: 4.2973e-04 - mse: 4.2973e-04 - val_loss: 0.0964 - val_mse: 0.0964
Epoch 125/150
33/33 [==============================] - 0s 4ms/step - loss: 3.8912e-04 - mse: 3.8912e-04 - val_loss: 0.0926 - val_mse: 0.0926
Epoch 126/150
33/33 [==============================] - 0s 4ms/step - loss: 2.8873e-04 - mse: 2.8873e-04 - val_loss: 0.0963 - val_mse: 0.0963
Epoch 127/150
33/33 [==============================] - 0s 4ms/step - loss: 2.9520e-04 - mse: 2.9520e-04 - val_loss: 0.0934 - val_mse: 0.0934
Epoch 128/150
33/33 [==============================] - 0s 4ms/step - loss: 2.2338e-04 - mse: 2.2338e-04 - val_loss: 0.0953 - val_mse: 0.0953
Epoch 129/150
33/33 [==============================] - 0s 4ms/step - loss: 1.6232e-04 - mse: 1.6232e-04 - val_loss: 0.0958 - val_mse: 0.0958
Epoch 130/150
33/33 [==============================] - 0s 5ms/step - loss: 1.6397e-04 - mse: 1.6397e-04 - val_loss: 0.0957 - val_mse: 0.0957
Epoch 131/150
33/33 [======

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [35]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0019 - mse: 0.0019


[0.001948997494764626, 0.001948997494764626]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [36]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.0939 - mse: 0.0939


[0.09390998631715775, 0.09390998631715775]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [37]:
# Evaluate the best model on test data
adam_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 2ms/step - loss: 0.1494 - mse: 0.1494


[0.1494472175836563, 0.1494472175836563]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [40]:
# Generate predictions on test data
y_test_pred_scaled = adam_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

30379.13327966274

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.